In [4]:

'''

GUILHERME LIMA DE SOUSA - 
ESTUDO DE CASO: TONELADAS TRANSPORTADAS EM FERROVIAS BRASILEIRAS
OBJETIVO: PREDIÇÃO USANDO RANDOMFORESTREGRESSOR

ENTRE EM CONTATO COMIGO NO LINKEDIN: 
www.linkedin.com/in/guilherme-lima-747355169

'''


import pandas as pd
import numpy as np
import glob
import os
import matplotlib.pyplot as plt
import seaborn as sns
from numpy import random
import mlflow


################################################################################


# ETL DA BASE

################################################################################



##CONFIGURANDO NUMEROS COM 2 CASAS DECIMAIS
pd.set_option('Float_format','{:.2f}'.format)

## COMPILANDO ARQUIVOS DA PASTA
pasta = 'C:/Users/Guilh/OneDrive/Área de Trabalho/1-MATERIAIS DE ESTUDO/01 - DIVERSOS/01-SCIKIT_LEARN/02-RANDOMFOREST/1-ANTT_FERROVIAS/01-INPUT'
arquivos = glob.glob(os.path.join(pasta, '*.csv'))
base_original = pd.concat([pd.read_csv(i, sep=';', encoding='latin1') for i in arquivos], ignore_index = True)

## REMOVENDO PONTO DA COLUNA
base_original['TU'] = base_original['TU'].str.replace('.','',regex=False)
base_original['TKU'] = base_original['TKU'].str.replace('.','',regex=False)

## ALTERANDO FORMATAÇÃO DA COLUNA
base_original['TKU'] = base_original['TKU'].fillna(0)
base_original['TKU'] = base_original['TKU'].astype(float)
base_original['TU'] = base_original['TU'].astype(int)
base_original['TKU'] = base_original['TKU'].astype(int)
base_original['Mes_Ano'] = pd.to_datetime(base_original['Mes_Ano'], format='%m/%Y')

## CRIANDO COLUNAS
base_original['MÊS'] = base_original['Mes_Ano'].dt.month
base_original['ANO'] = base_original['Mes_Ano'].dt.year

#REMOVENDO DUPLICADOS E NULOS
base_original = base_original.drop_duplicates()

# REMOVENDO COLUNA TKU POIS O OBJETIVO É APNAS TU
base_original = base_original.drop('TKU', axis=1)
base_original = base_original.drop('Mes_Ano', axis=1)

# REMOVENDO VALORES ABAIXO DE 1 TONELADA
base_original = base_original[base_original["TU"]>= 1]

# VENDO A BASE
base_original.head(-5)

,Ferrovia,Mercadoria_ANTT,Estacao_Origem,UF_Origem,Estacao_Destino,UF_Destino,TU,MÊS,ANO
0,EFC,Álcool,Itaqui Base Combustível,MA,Marabá,PA,184,1,2006
1,EFC,Bebidas e Vasilhames,Ponta da Madeira Pêra do Píer,MA,Imperatriz,MA,1636,1,2006
2,EFC,Cobre,Paraupebas,PA,Ponta da Madeira Cobre,MA,24461,1,2006
3,EFC,Ferro Gusa,Açailândia,MA,Ponta da Madeira Pêra do Píer,MA,116272,1,2006
4,EFC,Ferro Gusa,Marabá,PA,Ponta da Madeira Pêra do Píer,MA,205242,1,2006
...,...,...,...,...,...,...,...,...,...
148653,RMS,Soja,Marialva,PR,D Pedro II,PR,179173,4,2023
148654,RMS,Soja,Marialva,PR,São Francisco do Sul,SC,80527,4,2023
148655,RMS,Soja,Maringa,PR,D Pedro II,PR,193917,4,2023
148656,RMS,Soja,Maringa,PR,Rio Grande,RS,1710,4,2023


In [22]:


####################################################################################

#     CRIANDO DATASET DO MODELO

####################################################################################

'''

AQUI VAMOS CRIAR O DATASET DO MODELO
FAZENDO ETL COM AS COLUNAS CATEGORICAS E APLICANDO CÓDIGOS PARA CADA UMA, 
POSTERIORMENTE USANDO AS COLUNAS NUMERICAS REPRESENTADAS COMO VARIAVEIS DO MODELO.
EM VEZ DE USAR O NOME DA FERROVIA ESTAMOS CRIANDO UM CÓDIGO DISTINTO DELA.

'''


# REPRODUTIBILIDADE
np.random.seed(2)


## BASE MODELO
#print(base_original.columns)
print('--------------------')
base_modelo = base_original

# TRANSFORMANDO COLUNAS
base_modelo['ANO'] = base_modelo['ANO'].astype(int)
base_modelo['MÊS'] = base_modelo['MÊS'].astype(int)
base_modelo['TU'] = base_modelo['TU'].astype(int)

#
############### CRIANDO OS CÓDIGOS DE CADA CATEGORIA
# SELECIONA APENAS AS COLUNAS STRING/CATEGORIAS/OBJETO
categ = base_modelo.select_dtypes(include=['object', 'category']).copy()
# LAMBDA QUE TRANSFORMA AS COLUNAS E NUMEROS
cod_categorica_hoje_1 = categ.apply(lambda col: col.astype('category').cat.codes)
# RENOMEANDO AS COLUNAS DE CODIGO COM PREFIXO
cod_categorica_1 = cod_categorica_hoje_1.add_prefix('Codigo_')


# CONCATENANDO AS BASES DE CODIGOS COM A ORIGINAL
codigos_categorias = pd.concat([base_modelo, cod_categorica_1], axis=1)

print('################# BASE COM DE:PARA ########################\n')
# BASE COM TUDO



##########  PARA VISUALIZAR DE:PARA DE COLUNA ESPECIFICA
#visualizar_de_para = codigos_categorias[['Ferrovia', 'Codigo_Ferrovia']].drop_duplicates().reset_index(drop=True)
#print('\n################# VERIFICAR ########################\n')
#print(visualizar_de_para.head(3))


########## ENGENHARIA DE ATRIBUTOS - FEATURE ENGINEERING - CLUSTER KMEANS
grupos_produtos = pd.read_csv('BASE_ANTT_COM_KMEANS.csv', sep=';')
grupos_produtos = grupos_produtos.iloc[:,[2,3,4,5,6,7,9]].drop_duplicates()


mesclando_df = codigos_categorias.merge(grupos_produtos,how="left", on=['Codigo_Ferrovia',
                                                                     'Codigo_Mercadoria_ANTT',
                                                                     'Codigo_Estacao_Origem',
                                                                     'Codigo_UF_Origem',
                                                                     'Codigo_Estacao_Destino',
                                                                     'Codigo_UF_Destino'])

codigos_categorias = mesclando_df
codigos_categorias.head(5)


--------------------
################# BASE COM DE:PARA ########################



,Ferrovia,Mercadoria_ANTT,Estacao_Origem,UF_Origem,Estacao_Destino,UF_Destino,TU,MÊS,ANO,Codigo_Ferrovia,Codigo_Mercadoria_ANTT,Codigo_Estacao_Origem,Codigo_UF_Origem,Codigo_Estacao_Destino,Codigo_UF_Destino,cluster_produto
0,EFC,Álcool,Itaqui Base Combustível,MA,Marabá,PA,184,1,2006,0,98,216,5,234,9,NaN
1,EFC,Bebidas e Vasilhames,Ponta da Madeira Pêra do Píer,MA,Imperatriz,MA,1636,1,2006,0,11,316,5,187,5,1.00
2,EFC,Cobre,Paraupebas,PA,Ponta da Madeira Cobre,MA,24461,1,2006,0,22,293,9,296,5,0.00
3,EFC,Cobre,Paraupebas,PA,Ponta da Madeira Cobre,MA,24461,1,2006,0,22,293,9,296,5,2.00
4,EFC,Ferro Gusa,Açailândia,MA,Ponta da Madeira Pêra do Píer,MA,116272,1,2006,0,44,36,5,297,5,0.00


In [23]:
####################################################################################

#     TRANSFORMANDO DATASET DO MODELO RANDOMFOREST

####################################################################################

''' 
AQUI ESTOU REMOVENDO OS VALORES MENORES DE 1000 MIL TONELADAS DE MOVIMENTAÇÃO
POIS ABAIXO DE MIL DEIXA O MODELO RUIM, MUITO COMPLEXO PRA ENTENDER OS DADOS
QUE VARIAM DE ZERO A MILHOES.

'''
# CRIANDO A BASE
base_treino = codigos_categorias.select_dtypes(exclude=['object', 'category']).copy()

#REMOVENDO DUPLICADOS
base_treino = base_treino.drop_duplicates()
base_treino = base_treino.fillna(0)

# SALVANDO BASE DO MODELO COMPLETA SEM FILTROS
base_modelo_completa = base_treino

# LIMITANDO A BASE REMOVENDO 2023
base_treino = base_treino[base_treino['ANO'] != 2023][list(base_treino.columns)]


# REMOVENDO OS OUTLIERS - TUDO QUE É MENOR DE MIL TONELADAS
base_treino = base_treino[base_treino['TU'] >= 1000.00]
base_modelo = base_treino


# REMOVENDO OS OUTLIERS - TUDO QUE ESTÁ MAIOR DO 3º QUARTIL EM TONELADAS
#q3 = base_treino['TU'].quantile(0.75)
#base_treino = base_treino[base_treino['TU'] <= q3]
#base_modelo = base_treino



# LOGARITMO BASE10
base_modelo['TU_LOG10'] = np.log10(base_modelo['TU'])


# REMOVENDO COLUNA REAL
base_modelo = base_modelo.drop('TU', axis=1)


base_modelo = base_modelo[["MÊS", "ANO", "Codigo_Ferrovia", "Codigo_Mercadoria_ANTT",
                          "Codigo_Estacao_Origem", "Codigo_UF_Origem", "Codigo_Estacao_Destino",
                          "Codigo_UF_Destino", "cluster_produto", "TU_LOG10"]]


print('################# BASE MODELO ########################')
print(base_modelo.head(3))
base_modelo.describe()


################# BASE MODELO ########################
   MÊS   ANO  Codigo_Ferrovia  Codigo_Mercadoria_ANTT  Codigo_Estacao_Origem  \
1    1  2006                0                      11                    316   
2    1  2006                0                      22                    293   
3    1  2006                0                      22                    293   

   Codigo_UF_Origem  Codigo_Estacao_Destino  Codigo_UF_Destino  \
1                 5                     187                  5   
2                 9                     296                  5   
3                 9                     296                  5   

   cluster_produto  TU_LOG10  
1             1.00      3.21  
2             0.00      4.39  
3             2.00      4.39  


,MÊS,ANO,Codigo_Ferrovia,Codigo_Mercadoria_ANTT,Codigo_Estacao_Origem,Codigo_UF_Origem,Codigo_Estacao_Destino,Codigo_UF_Destino,cluster_produto,TU_LOG10
count,124771.00,124771.00,124771.00,124771.00,124771.00,124771.00,124771.00,124771.00,124771.00,124771.00
mean,6.55,2013.40,6.89,52.05,231.22,10.18,219.09,11.61,0.54,4.02
std,3.44,4.97,3.95,30.80,127.77,5.38,121.51,5.64,0.99,0.67
min,1.00,2006.00,0.00,0.00,1.00,0.00,0.00,0.00,0.00,3.00
25%,4.00,2009.00,3.00,23.00,128.00,6.00,120.00,6.00,0.00,3.51
50%,7.00,2013.00,7.00,52.00,242.00,9.00,228.00,13.00,0.00,3.92
75%,10.00,2018.00,12.00,81.00,341.00,15.00,331.00,18.00,1.00,4.41
max,12.00,2022.00,12.00,101.00,437.00,19.00,410.00,19.00,3.00,7.14


In [31]:

'''

HIPÓTESE NULA = SERÁ QUE MEU MODELO CONSEGUE EXPLICAR AS TONELADAS TRANSPORTADAS
APENAS COM AS DEMAIS COLUNAS DO NOSSO DATASET ?


############# PARAMETROS DO MODELO
MODELO COM VALIDAÇÃO CRUZADA DE 5 FOLDS CRIADO COM OS PARAMETROS:

N_ESTIMATORS = 300
300 ARVORES DE DECISÃO NO MÁXIMO

MAXDEPTH = 15
PERMITE CRIAR ARVORES MAIS PROFUNDAS E TRAZ ALEATORIEDADE AOS DADOS, GENERALIZAÇÃO.

RANDOM_STATE = 2
REPLICABILIDADE DE RESULTADOS

MAX_FEATURES = 'SQRT' 
ELE NÃO USA TODAS AS FEATURES NOS NÓS, ELE USA A RAIZ QUADRADA DO TOTAL DE COLUNAS.
DESTA FORMA O MODELO É FORÇADO COM DADOS E CENÁRIOS COMPLEXOS.

MIN_SAMPLES_SPLIT = 10
O MODELO SÓ VAI CRIAR OUTRO NÓ SE TIVER PELO MENOS 10 REGISTROS PRA SEGUIR ADIANTE

MIN_SAMPLES_LEAF = 5
CADA FOLHA FINAL DEVE CONTER PELO MENOS 5 REGISTROS


############# AVALIAÇÕES
# MAE (ERRO ABSOLUTO MEDIO)
Em média, o modelo erra cerca de 38.327 toneladas na previsão de TU. Esse valor deve ser 
interpretado em relação à escala dos dados — se TU varia de 0 a milhões, esse erro pode ser tolerável; se não, 
pode ser alto ......Em nossa base a coluna TARGET varia entre 0 e 13.858.719 então 38.327 é tolerável

# MSE (MEAN SQUARED ERROR) - Média dos erros ao quadrado. 
Penaliza fortemente grandes erros e destaca discrepâncias mais severas.

# MAPE
Este é o erro médio (MAE) só que em percentual.

# RMSE
Raiz do erro quadratico médio - (Raise mean squared error)
Esse erro penaliza mais fortemente grandes erros. Um RMSE muito maior que o MAE (como no nosso cenario 274.067) indica que 
há outliers ou erros muito grandes em alguns casos que estão influenciando bastante a performance.

# MEDAE
Este é a mediana dos erros, mostra como os erros estão distribuidos, o meio entre eles.

Avaliamos:
MAE para entender o erro médio bruto.
MAPE é o MAE em %.
RMSE se quiser punir grandes erros, fica sempre maior mas não tanto.
MedAE onde é o meio ? mediana dos erros.

'''


############################################################################


#  MODELO PRINCIPAL

############################################################################

from sklearn.ensemble import RandomForestClassifier, RandomForestRegressor
from sklearn.model_selection import KFold, cross_val_score, train_test_split, learning_curve, TimeSeriesSplit
from sklearn.metrics import confusion_matrix, ConfusionMatrixDisplay, accuracy_score, f1_score, classification_report
from sklearn.metrics import r2_score, mean_absolute_error, mean_squared_error, mean_absolute_percentage_error, median_absolute_error
from sklearn.datasets import make_regression
from sklearn.dummy import DummyRegressor


## DEFININDO MLFLOW
mlflow.set_tracking_uri("http://127.0.0.1:5000/")
mlflow.set_experiment(experiment_id=540394382883761644)


# REPRODUTIBILIDADE
np.random.seed(2)

base_modelo = base_modelo.sort_values(by=["ANO", "MÊS"])
########################  CRIANDO MODELO PRINCIPAL
# DEFININDO X (VAR DEPENDENTE) Y (VAR TARGET)
x = base_modelo.drop(columns=['TU_LOG10'])
y = base_modelo['TU_LOG10']


# ORDENAÇÃO DOS DADOS PARA A BASE DE TREINO
x = x[['MÊS', 'ANO', 'Codigo_Ferrovia', 'Codigo_Mercadoria_ANTT', 'Codigo_Estacao_Origem',
                                           'Codigo_UF_Origem', 'Codigo_Estacao_Destino', 'Codigo_UF_Destino',"cluster_produto"]]


# SEPARANDO TREINO E TESTE
x_train, x_test, y_train, y_test = train_test_split(x,y,test_size=0.1)



with mlflow.start_run():
    mlflow.sklearn.autolog()
    # CRIANDO MODELO
    modelo = RandomForestRegressor(n_estimators=300, 
                                max_depth=20, 
                                random_state=2, 
                                max_features='sqrt', 
                                min_samples_split=10, 
                                min_samples_leaf=5)

    # VALIDAÇÃO CRUZADA
    kf = KFold(n_splits=5, shuffle= True, random_state=2)
    cross_validation = cross_val_score(modelo, x, y, cv=kf, scoring='r2')

    # APLICANDO LOG() PARA MELHORAR A DISTRIBUIÇÃO DO MODELO - evita heterocedasticidade
    #y_train_log = np.log1p(y_train)


    # TREINANDO O MODELO
    modelo.fit(x_train, y_train)


    # USANDO MODELO NO COJUNTO DE TESTE - PREDIÇÃO
    y_pred = modelo.predict(x_test)


    # OBTENDO R2 DA PREDIÇÃO ANTERIOR
    ACC_MODELO = r2_score(y_test, y_pred)

    # ERRO PERCENTUAL MÉDIO COM LOG10
    MAE_LOG10 = mean_absolute_error(y_test, y_pred)
    MSE_LOG10 = mean_squared_error(y_test, y_pred)
    RMSE_LOG10 = np.sqrt(mean_squared_error(y_test, y_pred))
    MAPE_LOG10 = mean_absolute_percentage_error(y_test, y_pred)
    MEDAE_LOG10 = median_absolute_error(y_test, y_pred)


    # ERRO PERCENTUAL MÉDIO SEM LOG10
    MAE_REAL = mean_absolute_error(10**y_test, 10**y_pred)
    MSE_REAL = mean_squared_error(10**y_test, 10**y_pred)
    RMSE_REAL = np.sqrt(mean_squared_error(10**y_test, 10**y_pred))
    MAPE_REAL = mean_absolute_percentage_error(10**y_test, 10**y_pred)
    MEDAE_REAL = median_absolute_error(10**y_test, 10**y_pred)


    ##########  DEFININDO AS METRICAS QUE VAO PARA O MLFLOW
    metricas_mlflow = {
    "MAE_LOG10": MAE_LOG10,
    "MSE_LOG10": MSE_LOG10,
    "RMSE_LOG10": RMSE_LOG10,
    "MAPE_LOG10": MAPE_LOG10,
    "MEDAE_LOG10": MEDAE_LOG10,

    "MAE_REAL": MAE_REAL,
    "MSE_REAL": MSE_REAL,
    "RMSE_REAL": RMSE_REAL,
    "MAPE_REAL": MAPE_REAL,
    "MEDAE_REAL": MEDAE_REAL,
    "ACC_MODELO": ACC_MODELO}

    mlflow.log_metrics(metricas_mlflow)

2026/02/01 18:29:45 WARNING mlflow.utils.autologging_utils: MLflow autologging encountered a warning: "c:\Users\Guilh\AppData\Local\Programs\Python\Python313\Lib\site-packages\mlflow\types\utils.py:452: UserWarning: Hint: Inferred schema contains integer column(s). Integer columns in Python cannot represent missing values. If your input data contains missing values at inference time, it will be encoded as floats and will cause a schema enforcement error. The best way to avoid this problem is to infer the model schema based on a realistic data sample (training dataset) that includes missing values. Alternatively, you can declare integer columns as doubles (float64) whenever these columns may have missing values. See `Handling Integers With Missing Values <https://www.mlflow.org/docs/latest/models.html#handling-integers-with-missing-values>`_ for more details."
2026/02/01 18:30:13 WARNING mlflow.utils.autologging_utils: MLflow autologging encountered a warning: "c:\Users\Guilh\AppData\Lo

🏃 View run peaceful-crab-471 at: http://127.0.0.1:5000/#/experiments/540394382883761644/runs/7caa658643b24316be6b11298dcb943f
🧪 View experiment at: http://127.0.0.1:5000/#/experiments/540394382883761644
